In [1]:
#Import packages

#Rdkit
from rdkit import Chem

#Scipy
from scipy import stats 
from scipy.stats import uniform,ranksums 

#Pandas
import pandas as pd
from pandas import DataFrame

#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib #jbolib模块
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [2]:
import torch
import deepchem as dc
import numpy as np

In [3]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [4]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\HSV-3")

In [5]:
df= pd.read_csv('./pubchem_targettaxid_10335_bioactivity.csv',index_col=0)

In [6]:

df.dropna(subset=["PUBCHEM_CID"], inplace=True)  
df['Activity']=np.where(df['PUBCHEM_ACTIVITY_OUTCOME']=='Active', 1, 0)
df.sort_values(by="Activity" , inplace=True, ascending=False) 
df.drop_duplicates(subset='PUBCHEM_CID', keep='first', inplace=True)
df['cid']=df['PUBCHEM_CID']
df.index=df['cid']
df=df.drop(['PUBCHEM_CID'], axis=1)
df=df.drop(['PUBCHEM_SID'], axis=1)
df=df.drop(['cid'], axis=1)


KeyError: ['PUBCHEM_CID']

In [ ]:
df['IsomericSMILES']=df['PUBCHEM_EXT_DATASOURCE_SMILES']
Original_data=pd.DataFrame(df['IsomericSMILES'])
Original_data['Activity']=df['Activity']
Chemdraw_smiles=Original_data['IsomericSMILES'].tolist()

Original_data.head()

In [ ]:
Smis=[]
for i in Chemdraw_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Original_data['Canonical_smiles']=Canonical_smis
Original_data.to_csv("./Original_data.csv",sep=',',header=1,index=1)
Original_data

In [ ]:
idx= np.unique(Original_data.index.values, return_index = True )[1]
Original_data =Original_data.iloc[idx]
Original_data.to_csv("./Original_data.csv",sep=',',header=1,index=1)
Canonical_smis=Original_data['Canonical_smiles']
Original_data

In [7]:
Original_data =pd.read_csv('./Original_data.csv',index_col=0)
Canonical_smis=Original_data['Canonical_smiles']
Original_data

,Activite,IsomericSMILES,Canonical_smiles
cid,,,
3011383,1,C1=C(N=C(N=C1OC[C@H](CO)OCP(=O)(O)O)N)N,Nc1cc(OC[C@H](CO)OCP(=O)(O)O)nc(N)n1
53357718,1,C1=CC=C(C=C1)CC(CP(=O)(O)O)OCCN2C=NC3=C(N=CN=C...,Nc1ncnc2c1ncn2CCOC(Cc1ccccc1)CP(=O)(O)O
135398513,1,C1=NC2=C(N1COCCO)N=C(NC2=O)N,Nc1nc2c(ncn2COCCO)c(=O)[nH]1
446727,1,C1[C@@H]([C@H](O[C@H]1N2C=C(C(=O)NC2=O)/C=C/Br...,O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1...
168280858,1,CCN(CC)CC.CC(CN1C=NC2=C1N=C(NC2=O)N)(CO)OCP(=O...,CC(CO)(Cn1cnc2c(=O)[nH]c(N)nc21)OCP(=O)(O)O.CC...
...,...,...,...
60167465,0,C1CC1C#CC#CC2=CN(C(=O)NC2=O)[C@H]3C[C@@H]([C@H...,O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1...
60167322,0,C1[C@@H]([C@H](O[C@H]1N2C=C(C(=O)NC2=O)C#CC#CC...,Nc1cccc(C#CC#Cc2cn([C@H]3C[C@H](O)[C@@H](CO)O3...
70683866,0,CCCCCCCCOC1=NC(=NC2=C1N=CN2/C=C\3/CC3(CO)CO)N,CCCCCCCCOc1nc(N)nc2c1ncn2/C=C1/CC1(CO)CO


In [17]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [18]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis.tolist())
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_gelation=aDesc.get_output()

In [19]:
X_original_data=pd.DataFrame(Data_gelation)
Desc_names_names=aDesc.get_output_descriptors()
X_original_data.columns=Desc_names_names
X_original_data.index=Original_data.index

X_original_data.head()

,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,...,s1_numAroBonds,s2_numAroBonds,s3_numAroBonds,s4_numAroBonds,s34_size,s34_relSize,s34_phSize,s34_phRelSize,chiralMoment,chiralPhMoment
cid,,,,,,,,,,,,,,,,,,,,,
3011383,294.24,8.654118,20.4589,35.6472,21.0000,39.4645,0.601732,1.048447,0.617647,1.160721,...,0.0,0.0,0.0,6.0,16.0,0.842105,9.0,0.473684,15.362291,8.246211
53357718,377.38,8.203913,29.1043,46.8616,30.6195,52.3744,0.632702,1.018730,0.665641,1.138574,...,0.0,0.0,6.0,10.0,20.0,0.769231,6.0,0.230769,18.627936,9.949874
135398513,225.24,8.342222,16.8318,28.1417,16.6762,31.3653,0.623400,1.042285,0.617637,1.161678,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
446727,333.16,10.411250,20.8028,33.3708,21.2046,36.3764,0.650088,1.042838,0.662644,1.136762,...,0.0,0.0,0.0,0.0,16.0,0.842105,6.0,0.315789,18.906056,6.504435
168280858,434.50,7.241667,34.1893,61.0360,36.3412,69.3675,0.569822,1.017267,0.605687,1.156125,...,0.0,0.0,0.0,5.0,18.0,0.620690,10.0,0.344828,17.291616,9.539392


In [20]:
#Remove descriptors with excessive NA values
Thresh =int(0.8*len(X_original_data))
X_NAomit_data=X_original_data.dropna(
    #thresh=Thresh, 
    axis=1) 
X_NAomit_data.head()
X_NAomit_data.to_csv("./X_NAomit_data.csv",sep=',',header=1,index=1)

In [22]:
#Ranksum test
Ranksums_data=X_NAomit_data
Ranksums_data["Activity"]=Original_data['Activite']
Notgelable=Ranksums_data.loc[Ranksums_data["Activity"] == 0]
Gelable=Ranksums_data.loc[Ranksums_data["Activity"] == 1]
Ranksums_result=[]
for i in list(Ranksums_data.columns):
 NG = Notgelable[i]
 G = Gelable[i]
 Ranksums_caculate=ranksums(NG, G, alternative='two-sided')
 Ranksums_result.append(Ranksums_caculate)
Ranksums_results = pd.DataFrame(Ranksums_result, columns=['Statistics','Pvalues'])
Ranksums_results.index=Ranksums_data.columns#
Ranksums_results = Ranksums_results.drop('Activity',axis = 0)
Ranksums_results.sort_values(by="Pvalues" , inplace=True, ascending=True)
Ranksums_results

,Statistics,Pvalues
s4_pathLength,-12.249848,1.683101e-34
maxDistfromCC,-12.249132,1.698027e-34
meanDistFromCC,-11.886807,1.386049e-32
s4_size,-10.925797,8.677527e-28
s4_numRotBonds,-10.904136,1.101343e-27
...,...,...
B06[C-B],0.000000,1.000000e+00
B05[X-X],0.000000,1.000000e+00
B05[Si-X],0.000000,1.000000e+00
B05[B-X],0.000000,1.000000e+00


In [23]:
#Ranksum test <0.05
Ranksums_marked = Ranksums_results[Ranksums_results['Pvalues'] <0.05]
print(Ranksums_marked.shape)
Ranksums_marked.head()

(1422, 2)


,Statistics,Pvalues
s4_pathLength,-12.249848,1.683101e-34
maxDistfromCC,-12.249132,1.698027e-34
meanDistFromCC,-11.886807,1.386049e-32
s4_size,-10.925797,8.677527e-28
s4_numRotBonds,-10.904136,1.101343e-27


In [24]:
X_ranksums_data=X_NAomit_data[Ranksums_marked.index]
X_ranksums_data.head()

,s4_pathLength,maxDistfromCC,meanDistFromCC,s4_size,s4_numRotBonds,chiralMoment,s4_relPathLength,s4_relSize,s34_size,SHED_DL,...,piPC10,GATS8p,SM08_EA(bo),MATS7i,VE1sign_H2,CATS2D_04_AP,F05[N-O],NsssN,nCt,nRCOOR
cid,,,,,,,,,,,,,,,,,,,,,
3011383,6.000000,6.000000,3.500000,10.000000,2.000000,15.362291,0.315789,0.526316,16.0,3.789291,...,4.670490,0.973332,11.441977,-0.031017,0.282062,3.0,1.0,0.0,0.0,0.0
53357718,8.000000,8.000000,4.080000,13.000000,3.000000,18.627936,0.307692,0.500000,20.0,8.902572,...,6.341007,0.971842,12.535336,0.000302,0.063752,2.0,2.0,0.0,0.0,0.0
135398513,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,4.863198,1.459258,11.657172,0.003942,0.103398,2.0,1.0,0.0,0.0,0.0
446727,7.333333,7.333333,3.648148,12.333333,1.666667,18.906056,0.385965,0.649123,16.0,8.485281,...,4.465908,1.251605,10.765259,0.015442,0.091516,0.0,1.0,1.0,0.0,0.0
168280858,6.000000,6.000000,2.535714,12.000000,1.000000,17.291616,0.206897,0.413793,18.0,2.871746,...,5.809455,0.896664,12.024472,-0.158021,0.001552,2.0,2.0,1.0,1.0,0.0


In [25]:
#Scale data
Scaler = preprocessing.MinMaxScaler() #StandardScaler
Transformer =Scaler.fit(X_ranksums_data)
X_scaled_data=Transformer.transform(X_ranksums_data)
X_scaled_data =pd.DataFrame(X_scaled_data)
X_scaled_data.columns=X_ranksums_data.columns
X_scaled_data.index=Original_data.index
X_scaled_data.to_csv("./X_scaled_data.csv",sep=',',header=1,index=1)
joblib.dump(Transformer, './Scaler_transformer.pkl')
X_scaled_data.head()

,s4_pathLength,maxDistfromCC,meanDistFromCC,s4_size,s4_numRotBonds,chiralMoment,s4_relPathLength,s4_relSize,s34_size,SHED_DL,...,piPC10,GATS8p,SM08_EA(bo),MATS7i,VE1sign_H2,CATS2D_04_AP,F05[N-O],NsssN,nCt,nRCOOR
cid,,,,,,,,,,,,,,,,,,,,,
3011383,0.187500,0.187500,0.230070,0.161290,0.047619,0.150272,0.473684,0.574163,0.194574,0.206340,...,0.521670,0.302063,0.572597,0.553284,0.738502,0.75,0.090909,0.000000,0.000000,0.0
53357718,0.250000,0.250000,0.268196,0.209677,0.071429,0.182216,0.461538,0.545455,0.243218,0.484777,...,0.708258,0.301601,0.798497,0.591377,0.166918,0.50,0.181818,0.000000,0.000000,0.0
135398513,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.543194,0.452865,0.617059,0.595804,0.270720,0.50,0.090909,0.000000,0.000000,0.0
446727,0.229167,0.229167,0.239808,0.198925,0.039683,0.184936,0.578947,0.708134,0.194574,0.462054,...,0.498819,0.388422,0.432780,0.609791,0.239609,0.00,0.090909,0.142857,0.000000,0.0
168280858,0.187500,0.187500,0.166683,0.193548,0.023810,0.169144,0.310345,0.451411,0.218896,0.156377,...,0.648886,0.278270,0.692947,0.398806,0.004063,0.50,0.181818,0.142857,0.142857,0.0


In [26]:
#Remove high correlation descriptors
X_cor_data=trimm_correlated(X_scaled_data,0.80)
X_cor_data.index=Original_data.index
X_cor_data.to_csv("./ML_data.csv",sep=',',header=1,index=1)
print_RxC(X_cor_data)
X_cor_data.head()

1655 Moleculars × 194 Descriptors


,s4_pathLength,SHED_DL,mindO,TIE,phLevel3,NdssC,nROR,minssCH2,IC5,F05[C-O],...,B10[C-F],T(N..S),MATS1v,P_VSA_LogP_5,B08[C-C],GATS5m,GATS8p,MATS7i,nCt,nRCOOR
cid,,,,,,,,,,,,,,,,,,,,,
3011383,0.187500,0.206340,0.762786,0.021785,0.00,0.000000,0.076923,0.248516,0.678617,0.186047,...,0.0,0.0,0.685877,0.094285,0.0,0.703365,0.302063,0.553284,0.000000,0.0
53357718,0.250000,0.484777,0.820012,0.023288,0.75,0.000000,0.076923,0.405045,0.749968,0.209302,...,0.0,0.0,0.548521,0.135835,1.0,0.606525,0.301601,0.591377,0.000000,0.0
135398513,0.000000,0.000000,0.820461,0.009111,0.00,0.153846,0.076923,0.491773,0.636705,0.069767,...,0.0,0.0,0.403068,0.203738,0.0,0.656244,0.452865,0.595804,0.000000,0.0
446727,0.229167,0.462054,0.827658,0.023375,0.50,0.230769,0.076923,0.405772,0.717092,0.186047,...,0.0,0.0,0.528303,0.027305,1.0,0.481643,0.388422,0.609791,0.000000,0.0
168280858,0.187500,0.156377,0.783192,0.060976,0.00,0.153846,0.076923,0.238515,0.629374,0.302326,...,0.0,0.0,0.335561,0.230518,0.0,0.572332,0.278270,0.398806,0.142857,0.0
